# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [71]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots

In [72]:
include("../../../src/decompositions/naive_HOSVD.jl")
# include("../../../src/decompositions/CR_SVD.jl")

naive_HOSVD (generic function with 1 method)

### Load data and construct manifold ###

In [73]:
# load data
# ni = niread("data/nifti_dt.nii.gz") * 1e9;
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [74]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 2D slice 
data = predata[34:81,32:79,15];

# construct data manifold
n = size(data)
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M) # -> different than in 1D

6

In [75]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

### Construct low rank approximation ###

In [76]:
q = 1. * Matrix(I, 3, 3)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [77]:
N_R_q, N_U = naive_HOSVD(M,q, data);  # Error in power manifold representation
# CR_R_q, CR_U = CR_SVD(M,q, data);

(1,)
(48,)


LoadError: MethodError: no method matching symmetric_type(::Type{Vector{Matrix{Float64}}})
[0mClosest candidates are:
[0m  symmetric_type([91m::Type{T}[39m) where {S<:Number, T<:AbstractMatrix{S}} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:77
[0m  symmetric_type([91m::Type{T}[39m) where {S<:(AbstractMatrix), T<:AbstractMatrix{S}} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:80
[0m  symmetric_type([91m::Type{T}[39m) where {S, T<:AbstractMatrix{S}} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:74
[0m  ...

In [ ]:
ref_distance = sqrt(sum(distance.(Ref(M), Ref(q), data).^2))

N_tangent_distances_r = zeros(d)
N_distances_r = zeros(d)
CR_tangent_distances_r = zeros(d)
CR_distances_r = zeros(d)

for rank in 1:d
    N_log_q_data_r = Symmetric.([sum([N_R_q[i] * N_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    CR_log_q_data_r = Symmetric.([sum([CR_R_q[i] * CR_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    # expoentiate back
    N_data_r = exp.(Ref(M), Ref(q), N_log_q_data_r)
    CR_data_r = exp.(Ref(M), Ref(q), CR_log_q_data_r)


    # compute relative tangent space error
    N_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - N_log_q_data_r).^2)) / ref_distance
    CR_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - CR_log_q_data_r).^2)) / ref_distance


    # compute relative manifold error
    N_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, N_data_r).^2)) / ref_distance
    CR_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, CR_data_r).^2)) / ref_distance
end

In [ ]:
plot(1:d, [N_distances_r, CR_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/DTI_camino_1D_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/DTI_camino_1D_rank.png"